In [108]:
#-----IMPORTANDO ARQUIVO CSV E CRIANDO DATAFRAME 'df_aeronave'-----#

import os 
import pandas as pd
import numpy as np 
import psycopg2 as pg

df_aeronave = pd.read_csv('C:\\Users\\*****\\Desktop\\Projetos\\aeronave.csv', encoding='utf-8', delimiter=';')

In [113]:
#-----CONECTANDO AO BANCO DE DADOS-----#

host = 'localhost'
dbname = 'projeto_acidentes_aeroviarios'
user = '********'
password = '*******'
sslmode = 'disable'

conect_string = 'host={} user={} dbname={} password={} sslmode={}'.format(host, user, dbname, password, sslmode)
conec = pg.connect(conect_string)
cursor = conec.cursor()

print('Conectado ao banco de dados {} com sucesso'.format(dbname))

Conectado ao banco de dados projeto_acidentes_aeroviarios com sucesso


In [18]:
#-----GERANDO NOMES DAS COLUNAS PARA AUTOMATIZAR O COMANDO DE CRIAÇÃO DE TABELA-----#

tipos_de_dados_formatpostgr = {
    'object' : 'varchar',
    'int32' : 'integer',
    'int64' : 'integer',
    'datetime64[ns]' : 'date',
    'float64' : 'numeric'
}
colunas_formatadas = ', '.join('{} {}'.format(n, d) for (n, d) in zip(df_aeronave.columns, df_aeronave.dtypes.replace(tipos_de_dados_formatpostgr)))
colunas_aeronave = colunas_formatadas
colunas_aeronave

'codigo_ocorrencia2 integer, aeronave_matricula varchar, aeronave_operador_categoria varchar, aeronave_tipo_veiculo varchar, aeronave_fabricante varchar, aeronave_modelo varchar, aeronave_tipo_icao varchar, aeronave_motor_tipo varchar, aeronave_motor_quantidade varchar, aeronave_pmd numeric, aeronave_pmd_categoria numeric, aeronave_assentos numeric, aeronave_ano_fabricacao numeric, aeronave_pais_fabricante varchar, aeronave_pais_registro varchar, aeronave_registro_categoria varchar, aeronave_registro_segmento varchar, aeronave_voo_origem varchar, aeronave_voo_destino varchar, aeronave_fase_operacao varchar, aeronave_tipo_operacao varchar, aeronave_nivel_dano varchar, aeronave_fatalidades_total numeric'

In [110]:
#-----CRIANDO NOVA TABELA DENTRO DO BANCO DE DADOS-----#

nome_da_tabela = 'aeronave'
colunas = "codigo_ocorrencia2 integer, aeronave_matricula varchar, aeronave_operador_categoria varchar, aeronave_tipo_veiculo varchar, aeronave_fabricante varchar, aeronave_modelo varchar, aeronave_tipo_icao varchar, aeronave_motor_tipo varchar, aeronave_motor_quantidade varchar, aeronave_pmd integer, aeronave_pmd_categoria integer, aeronave_assentos integer, aeronave_ano_fabricacao integer, aeronave_pais_fabricante varchar, aeronave_pais_registro varchar, aeronave_registro_categoria varchar, aeronave_registro_segmento varchar, aeronave_voo_origem varchar, aeronave_voo_destino varchar, aeronave_fase_operacao varchar, aeronave_tipo_operacao varchar, aeronave_nivel_dano varchar, aeronave_fatalidades_total integer"
sql_querry = 'create table {} ({})'.format(nome_da_tabela, colunas)
cursor.execute(sql_querry)
conec.commit()
cursor.close()
conec.close()

In [111]:
#-----CORREÇÃO DE ERROS NA FORMATAÇÃO DOS DADOS-----#

#-----Células com dados inválidos
df_aeronave = df_aeronave.replace('***', '')
df_aeronave['aeronave_pmd'] = df_aeronave['aeronave_pmd'].astype(str).str.replace('nan', '', regex=True)
df_aeronave['aeronave_pmd_categoria'] = df_aeronave['aeronave_pmd_categoria'].astype(str).str.replace('nan', '', regex=True)
df_aeronave['aeronave_assentos'] = df_aeronave['aeronave_assentos'].astype(str).str.replace('nan', '', regex=True)
df_aeronave['aeronave_fatalidades_total'] = df_aeronave['aeronave_fatalidades_total'].astype(str).str.replace('nan', '', regex=True)
df_aeronave['aeronave_ano_fabricacao'] = df_aeronave['aeronave_ano_fabricacao'].astype(str).str.replace('nan', '', regex=True)

#-----Números inteiros com formatação de flutuantes
df_aeronave['aeronave_pmd_categoria'] = df_aeronave['aeronave_pmd_categoria'].astype(str).str.replace('.0', '', regex=True)
df_aeronave['aeronave_pmd'] = df_aeronave['aeronave_pmd'].astype(str).str.replace('.0', '', regex=True)
df_aeronave['aeronave_assentos'] = df_aeronave['aeronave_assentos'].astype(str).str.replace('.0', '', regex=True)
df_aeronave['aeronave_ano_fabricacao'] = df_aeronave['aeronave_ano_fabricacao'].astype(str).str.replace('.0', '', regex=True)
df_aeronave['aeronave_fatalidades_total'] = df_aeronave['aeronave_fatalidades_total'].astype(str).str.replace('.0', '', regex=True)

In [114]:
#-----SALVANDO O DATA FRAME 'df_aeronave' DENTRO DA TABELA 'aeronave' NO BANCO DE DADOS-----#

df_aeronave.to_csv('aeronave', header=df_aeronave.columns, index=False, encoding='utf-8')
arquivo_aberto_para_upload = open('aeronave', encoding='utf-8')

consulta_sql_para_upload = """
COPY aeronave FROM STDIN WITH
    CSV HEADER
    DELIMITER ','
    ENCODING 'utf-8'
"""

cursor.copy_expert(sql=consulta_sql_para_upload, file=arquivo_aberto_para_upload)
conec.commit()
cursor.close()
conec.close()

print('Arquivo adicionado a tabela {} no banco de dados {} com sucesso'.format(nome_da_tabela, dbname))

Arquivo adicionado a tabela aeronave no banco de dados projeto_acidentes_aeroviarios com sucesso
